In [1]:
import pandas as pd

In [2]:
import os

In [3]:
os.chdir("D:\Courses\CE 7090 -Statistical and Econometric Methods in Civil Engineering II\HomeWork\HW-4")

In [4]:
# Load data, treating -999 as NaN (missing)
df = pd.read_csv('HW4 Data.txt', delim_whitespace=True, header=None, na_values=-999)

In [5]:
# Basic inspection
print(df.shape)        # should be (999, 49)
print(df[[3,24,33]].head())  # peek at severity (X4), age (X25), speed limit (X34)

(999, 49)
   3   24  33
0   0  63  60
1   0   0  60
2   0  16  55
3   1  30  60
4   1  53  60


In [6]:
# Count missing values per column
missing_counts = df.isna().sum()
print(missing_counts[missing_counts > 0])


21    879
26     27
27     27
29     12
37    619
39    180
40    180
41    180
42    180
43    180
44    180
45    916
46    715
47    715
dtype: int64


In [7]:
# Drop columns with > 500 missing values (arbitrary threshold, e.g., X22, X46, X47, X48)
cols_to_drop = [col for col in df.columns if df[col].isna().sum() > 500]
df.drop(columns=cols_to_drop, inplace=True)

# Drop rows where age (X25) is missing or zero (assuming 0 means unknown age)
df = df[df[24] != 0]  # X25 (driver age) is at index 24

# Verify remaining shape after drops
print(df.shape)


(982, 44)


In [8]:
# Convert certain codes to categories or create dummies
df[5] = df[5].astype('category')        # X6 City (Seattle or not)
df[16] = df[16].astype('category')      # X17 Sobriety codes as category
df['winter'] = df[8].isin([12, 1, 2]).astype(int)      # X9 (month) in {12,1,2}
df['fix_obj'] = (df[17] == 50).astype(int)             # X18 collision type 50
df['young_driver'] = (df[24] <= 25).astype(int)        # X25 age <= 25


In [9]:
print(df[24].describe())    # driver age stats
print(df[3].value_counts()) # severity counts

count    982.000000
mean      34.983707
std       15.092570
min       16.000000
25%       23.000000
50%       30.000000
75%       45.000000
max       85.000000
Name: 24, dtype: float64
3
0    705
1    248
2     22
3      7
Name: count, dtype: int64


In [10]:
# Example: proportion of severe outcomes when no seatbelt vs seatbelt used
df['no_restraint'] = (df[27] == 3).astype(int)  # X28 code 3 = no restraints used
print(pd.crosstab(df['no_restraint'], df[3], normalize='index') * 100)


3                     0          1          2         3
no_restraint                                           
0             72.164948  25.257732   1.958763  0.618557
1             41.666667  25.000000  25.000000  8.333333


In [11]:
import statsmodels.api as sm

In [12]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [13]:
# Define outcome and predictors
y = df['severity'] = df[3]  # X4 Injury severity
X = df[['winter', 'fix_obj', 'young_driver', 'no_restraint']]  # example predictors
# Note: If needed, add others like alcohol or speed_limit. Here for illustration, using a few.

# Fit ordered probit (probit link for ordinal) 
model = OrderedModel(y, X, distr='probit')
result = model.fit(method='bfgs')
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.697894
         Iterations: 31
         Function evaluations: 32
         Gradient evaluations: 32
                             OrderedModel Results                             
Dep. Variable:                      3   Log-Likelihood:                -685.33
Model:                   OrderedModel   AIC:                             1385.
Method:            Maximum Likelihood   BIC:                             1419.
Date:                Sun, 16 Mar 2025                                         
Time:                        20:45:25                                         
No. Observations:                 982                                         
Df Residuals:                     975                                         
Df Model:                           4                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------

In [14]:
import numpy as np
from scipy.stats import norm

In [20]:
import numpy as np
from scipy.stats import norm
from statsmodels.miscmodels.ordinal_model import OrderedModel

max_iter = 100
tol_ll = 1e-4       # tolerance for log-likelihood change
tol_params = 1e-4   # tolerance for parameter change
prev_ll = -np.inf
N = len(df)
K = 2

# Randomly initialize responsibilities
np.random.seed(42)
r = np.zeros((N, K))
for i in range(N):
    r[i, 0] = np.random.uniform(0.4, 0.6)
    r[i, 1] = 1 - r[i, 0]

# Initialize placeholders for parameters
prev_class_params = [None] * K

for iteration in range(max_iter):
    # M-step: update model parameters for each class
    class_params = []
    class_thresholds = []
    for c in range(K):
        weights = r[:, c]
        model_c = OrderedModel(y, X, distr='probit')
        res_c = model_c.fit(method='bfgs', weights=weights, disp=False)
        # Assume last (model_c.k_levels - 1) parameters are thresholds
        n_thresholds = model_c.k_levels - 1
        params_c = res_c.params
        class_params.append(params_c[:-n_thresholds])
        class_thresholds.append(params_c[-n_thresholds:])
    
    # Update mixing proportions
    pi = r.mean(axis=0)
    
    # E-step: update responsibilities and compute log-likelihood
    ll_total = 0
    for i in range(N):
        L = np.zeros(K)
        for c in range(K):
            xb = np.dot(X.iloc[i], class_params[c])
            # Compute likelihood based on outcome (assuming 4 ordered outcomes: 0,1,2,3)
            if y.iloc[i] == 0:
                L[c] = norm.cdf(class_thresholds[c][0] - xb)
            elif y.iloc[i] == 1:
                L[c] = norm.cdf(class_thresholds[c][1] - xb) - norm.cdf(class_thresholds[c][0] - xb)
            elif y.iloc[i] == 2:
                L[c] = norm.cdf(class_thresholds[c][2] - xb) - norm.cdf(class_thresholds[c][1] - xb)
            elif y.iloc[i] == 3:
                L[c] = 1 - norm.cdf(class_thresholds[c][-1] - xb)
            L[c] = max(L[c], 1e-8)  # avoid zeros
        
        total = np.sum([pi[c] * L[c] for c in range(K)])
        ll_total += np.log(total)
        for c in range(K):
            r[i, c] = (pi[c] * L[c]) / total

    # Check convergence on log-likelihood
    ll_change = np.abs(ll_total - prev_ll)
    
    # Check convergence on parameter estimates if not first iteration
    param_change = 0
    if iteration > 0:
        for c in range(K):
            diff = np.abs(class_params[c] - prev_class_params[c])
            param_change = max(param_change, np.max(diff))
    
    print(f"Iteration {iteration}: Log-likelihood = {ll_total:.4f}, ll_change = {ll_change:.6f}, param_change = {param_change:.6f}")
    
    if ll_change < tol_ll and param_change < tol_params:
        print(f"Converged at iteration {iteration}")
        break
    
    prev_ll = ll_total
    prev_class_params = class_params.copy()

print("Final Mixing Proportions:", pi)
for c in range(K):
    print(f"\nClass {c+1} Coefficients:")
    print(class_params[c])
    print(f"Class {c+1} Thresholds:")
    print(class_thresholds[c])


Iteration 0: Log-likelihood = -5208.9729, ll_change = inf, param_change = 0.000000
Iteration 1: Log-likelihood = -5208.9729, ll_change = 0.000000, param_change = 0.000000
Converged at iteration 1
Final Mixing Proportions: [0.49804541 0.50195459]

Class 1 Coefficients:
winter         -0.173256
fix_obj        -0.074944
young_driver    0.062003
no_restraint    1.096298
dtype: float64
Class 1 Thresholds:
0/1    0.500270
1/2    0.291836
2/3   -0.520236
dtype: float64

Class 2 Coefficients:
winter         -0.173256
fix_obj        -0.074944
young_driver    0.062003
no_restraint    1.096298
dtype: float64
Class 2 Thresholds:
0/1    0.500270
1/2    0.291836
2/3   -0.520236
dtype: float64


In [21]:
# After the EM loop has finished:
print("Final Mixing Proportions:")
for c in range(K):
    print(f"Class {c+1}: {pi[c]:.4f}")

print("\nFinal Model Estimates for Each Class:")
for c in range(K):
    print(f"\nClass {c+1} Coefficients:")
    print(class_params[c])
    print(f"Class {c+1} Thresholds:")
    print(class_thresholds[c])

# Optionally, assign each observation to a class based on the highest responsibility:
df['predicted_class'] = np.argmax(r, axis=1) + 1  # +1 to make classes 1-indexed
print("\nDistribution of Observations by Predicted Class:")
print(df['predicted_class'].value_counts())


Final Mixing Proportions:
Class 1: 0.4980
Class 2: 0.5020

Final Model Estimates for Each Class:

Class 1 Coefficients:
winter         -0.173256
fix_obj        -0.074944
young_driver    0.062003
no_restraint    1.096298
dtype: float64
Class 1 Thresholds:
0/1    0.500270
1/2    0.291836
2/3   -0.520236
dtype: float64

Class 2 Coefficients:
winter         -0.173256
fix_obj        -0.074944
young_driver    0.062003
no_restraint    1.096298
dtype: float64
Class 2 Thresholds:
0/1    0.500270
1/2    0.291836
2/3   -0.520236
dtype: float64

Distribution of Observations by Predicted Class:
predicted_class
2    982
Name: count, dtype: int64
